In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from sklearn.datasets import fetch_20newsgroups

newsgroups = fetch_20newsgroups()

In [ ]:
newsgroups.keys()

In [ ]:
#print(newsgroups.DESCR)

In [ ]:
newsgroups.target_names

In [ ]:
print(newsgroups.data[0])

In [ ]:
newsgroups.target_names[newsgroups.target[0]]

In [ ]:
categories = ['talk.politics.guns', 'talk.religion.misc', 'comp.graphics', 'rec.autos']
newsgroups = fetch_20newsgroups(categories=categories)
target_names = newsgroups.target_names

In [ ]:
newsgroups.target.shape

In [ ]:
print(newsgroups.data[0])

In [ ]:
target_names[newsgroups.target[0]]

In [ ]:
from sklearn.model_selection import train_test_split

data_train, data_test, target_train, target_test = train_test_split(newsgroups.data,
                                                                    newsgroups.target,
                                                                    test_size=0.2,
                                                                    random_state=42)

In [ ]:
newsgroups.target.shape

In [ ]:
target_train.shape

In [ ]:
target_test.shape

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(['I am teapot', 'Short and stout'])
vectorizer.get_feature_names()

In [ ]:
pd.DataFrame(data=vectorizer.transform(['I am a little teapot', 'Short and short']).toarray(),
            columns=vectorizer.get_feature_names())

In [ ]:
tf_vectorizer = CountVectorizer()
tf_train = tf_vectorizer.fit_transform(data_train)
tf_train.shape

In [ ]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf.fit(tf_train, target_train)

In [ ]:
from sklearn import metrics

pred = clf.predict(tf_train)
metrics.accuracy_score(target_train, pred)

In [ ]:
pred = clf.predict(tf_vectorizer.transform(data_test))
metrics.accuracy_score(target_test, pred)

In [ ]:
def show_most_informative_features(classifier, vectorizer, categories):
    for i, c in enumerate(categories):
        print("%s: %s" % (c, ", ".join(np.asarray(vectorizer.get_feature_names())[np.argsort(classifier.coef_[i])[-10:]])))

In [ ]:
show_most_informative_features(clf, tf_vectorizer, target_names)

In [ ]:
from sklearn.pipeline import make_pipeline

model = make_pipeline(
    CountVectorizer(stop_words='english'),
    MultinomialNB())
model.fit(data_train, target_train)

In [ ]:
show_most_informative_features(
    model.named_steps['multinomialnb'],
    model.named_steps['countvectorizer'],
    target_names)

In [ ]:
model.score(data_test, target_test)

In [ ]:
newsgroups_clean = fetch_20newsgroups(categories=categories, remove=('headers', 'footers', 'quotes'))
print(newsgroups_clean.data[0])

In [ ]:
data_train_clean, data_test_clean, target_train_clean, target_test_clean = train_test_split(
    newsgroups_clean.data,
    newsgroups_clean.target,
    test_size=0.2,
    random_state=42)

In [ ]:
print(data_test_clean[0])
print(target_test_clean[0])
print(target_names[target_test_clean[0]])

In [ ]:
model.predict([data_test_clean[0]])

In [ ]:
model.score(data_test_clean, target_test_clean)

In [ ]:
#pred = model.predict(data_test_clean)
#metrics.f1_score(target_test_clean, pred, average='macro')

In [ ]:
model = make_pipeline(
    CountVectorizer(stop_words='english'),
    MultinomialNB())
model.fit(data_train_clean, target_train_clean)

In [ ]:
model.score(data_test_clean, target_test_clean)

In [ ]:
show_most_informative_features(
    model.named_steps['multinomialnb'],
    model.named_steps['countvectorizer'],
    target_names)

In [ ]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
tf_vectorizer = CountVectorizer(
    stop_words = 'english',
    #token_pattern = r'\b[a-zA-Z]{3,}\b',
    #max_df = 0.5, 
    #min_df = 10
)
tf = tf_vectorizer.fit_transform(newsgroups_clean.data)
lda = LatentDirichletAllocation(n_components=10, random_state=42)
lda.fit(tf)
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

In [ ]:
target_names